# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.common import ML_api
influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


cuda is available.
cuda is available.


# 1. set param from Front End

In [2]:
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        #step_split일 경우만 past_step과 future_step이 존재
        'data_clean_option' : "false"
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'step_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        'past_step':24, #step_split일 경우만 past_step과 future_step이 존재
        'future_step':2
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainy',
        "feature_list":['value']
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'window_split' # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
    },
    "model_info" :{
        'model_purpose' : 'classification',
        'model_method' : 'LSTM_cf',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True","num_classes":6, 'rnn_type':'lstm'}
    }
}

# 2. Backend

In [14]:
# 1. Backend Prameter Setting
from Clust.clust.ML.common import ML_api

# parameter tunning
param = param1
param = ML_api.convert_param_for_backend(param)

In [15]:
# 2. train data meta check
train_data_info = ML_api.get_train_data_meta (mongodb_client, param['ingestion_param_X'])
param['train_data_info'] = train_data_info

In [16]:
# 3. Data Preparation
#train_X_array, train_y_array, val_X_array, val_y_array = ML_api.ML_data_preparation(param, influxdb_client)

from Clust.clust.ML.common import ML_pipeline
# 1. Oirignla data ingestion
data_X, data_y = ML_pipeline.Xy_data_preparation(param['ingestion_param_X'], 
                                             param['data_y_flag'], 
                                             param['ingestion_param_y'],
                                             'ms_all', 
                                             influxdb_client)
# 2. Scaling
dataX_scaled, datay_scaled = ML_pipeline.Xy_data_scaling_train(param['ingestion_param_X']['ms_name'], 
                                                                                 data_X, 
                                                                                 param['ingestion_param_y']['ms_name'], 
                                                                                 data_y, 
                                                                                 param['scaler_param'])



# 3.clean column
dataX_scaled = ML_pipeline.clean_low_quality_column(dataX_scaled, param['transform_param'])

param['transform_param']

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_trainX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}
['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
./scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl
['value']
Make New scaler File


{'split_mode': 'window_split',
 'data_clean_option': False,
 'nan_process_info': {'type': 'num',
  'ConsecutiveNanLimit': 10000,
  'totalNaNLimit': 100000},
 'max_nan_limit_ratio': 0.5}

In [17]:
# 4. split train/Val
split_ratio = 0.8
train_X, val_X, train_y, val_y, param['transform_param']= ML_pipeline.split_data_by_mode(dataX_scaled, 
                                                                                         datay_scaled, 
                                                                                         split_ratio, 
                                                                                         param['transform_param'])

{'split_mode': 'window_split', 'data_clean_option': False, 'nan_process_info': {'type': 'num', 'ConsecutiveNanLimit': 10000, 'totalNaNLimit': 100000}, 'max_nan_limit_ratio': 0.5, 'future_step': None, 'past_step': 144}
window_split
None


In [18]:
train_X

,Press_mm_hg,RH_1,RH_2,RH_3,RH_4,RH_5,RH_6,RH_7,RH_8,RH_9,...,T4,T5,T6,T7,T8,T9,T_out,Tdewpoint,Visibility,Windspeed
time,,,,,,,,,,,,,,,,,,,,,
2016-01-11 00:00:00+00:00,0.693237,0.127786,0.324722,0.255336,0.102083,0.230779,0.247370,0.205674,0.171008,0.426523,...,0.417117,0.196847,0.353806,0.622997,0.570034,0.375650,0.335484,0.108597,0.609375,0.538462
2016-01-11 00:10:00+00:00,0.695250,0.134942,0.328622,0.258140,0.105062,0.228774,0.252495,0.203073,0.171008,0.428085,...,0.417117,0.196847,0.350216,0.616085,0.563625,0.375650,0.332258,0.110106,0.609375,0.525641
2016-01-11 00:20:00+00:00,0.697262,0.138611,0.336128,0.263281,0.112363,0.229225,0.249798,0.204374,0.173521,0.428085,...,0.414414,0.203854,0.346335,0.609488,0.556912,0.375650,0.329032,0.111614,0.609375,0.512821
2016-01-11 00:30:00+00:00,0.699275,0.144115,0.343927,0.263281,0.118024,0.228323,0.247808,0.204374,0.177862,0.426523,...,0.414414,0.196847,0.344297,0.603205,0.547757,0.371835,0.325806,0.113122,0.609375,0.500000
2016-01-11 00:40:00+00:00,0.701288,0.146133,0.350556,0.263281,0.124430,0.226819,0.249764,0.203073,0.175805,0.426523,...,0.411411,0.196847,0.342163,0.594722,0.542264,0.371835,0.322581,0.114630,0.609375,0.487179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-03-26 23:10:00+00:00,0.754831,0.201174,0.328037,0.253934,0.148118,0.121567,0.000000,0.192819,0.177862,0.391392,...,0.723724,0.704252,0.682724,0.791706,0.805005,0.832466,0.605376,0.479638,0.609375,0.153846
2016-03-26 23:20:00+00:00,0.753623,0.188148,0.320199,0.253934,0.138881,0.116991,0.000000,0.189970,0.174777,0.389749,...,0.717718,0.704252,0.682724,0.790494,0.802389,0.832466,0.607527,0.466063,0.609375,0.153846
2016-03-26 23:30:00+00:00,0.752415,0.185396,0.330209,0.253934,0.147820,0.111853,0.000000,0.190780,0.174777,0.386706,...,0.717718,0.704252,0.685926,0.786616,0.795850,0.832466,0.609677,0.452489,0.609375,0.153846


In [19]:
print(len(val_X)/128)
print(len(val_y))

21.375
19


In [20]:
# 5. Transform array style
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                        train_X, 
                                                                        train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                    val_X, 
                                                                    val_y)

window_size: 144 nan_limit_num: 72
(10944, 24) (72, 144, 24)
(76, 1) (72, 1)
window_size: 144 nan_limit_num: 72
(2736, 24) (19, 144, 24)
(19, 1) (19, 1)


In [21]:
# 4.Training
param = ML_api.ML_training(train_X_array,  train_y_array, val_X_array, val_y_array, param)

{'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}
./Models/LSTM_rg/regression_energy_cleanLevel4_trainX_regression_LSTM_rg_/regression_energy_cleanLevel4_trainX/model.pkl
Start training model


RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu

In [22]:
param

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}},


In [23]:
# save meta
from Clust.clust.ML.tool import meta
meta_file_name = "meta.json"
#ml_meta.save_model_meta_into_mongodb(mongodb_client, param, 'model','meta')
meta.save_model_meta_into_local(meta_file_name, param)

======== OK ========


200